In [1]:
# 1. Cài đặt các thư viện lõi (YOLO, PaddleOCR, Streamlit)
!pip install ultralytics paddlepaddle-gpu paddleocr==2.7.3 streamlit -q

# 2. [QUAN TRỌNG] Hạ cấp Numpy xuống bản 1.x để tương thích với PaddleOCR
!pip install "numpy<2.0.0" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [2]:
# Giải nén model
!tar -xvf yolov11-lp-nano-other-default-v1.tar.gz

tar: yolov11-lp-nano-other-default-v1.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [3]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from PIL import Image
from ultralytics import YOLO
from paddleocr import PaddleOCR
import pandas as pd
import os
import re
import time
import tempfile
from datetime import datetime

# --- CẤU HÌNH ---
MODEL_PATH = '/content/kaggle/working/runs/segment/yolo11_license_plate/weights/best.pt'
HISTORY_FILE = 'parking_history.csv'

st.set_page_config(page_title="AI Smart Parking", layout="wide", page_icon="📹")

st.markdown("""
    <style>
        .stButton>button {width: 100%;}
        .main {background-color: #f5f5f5;}
    </style>
""", unsafe_allow_html=True)

# ==========================================
# 1. BỘ LỌC THÔNG MINH (QUAN TRỌNG)
# ==========================================
def clean_text(text):
    """Chỉ giữ lại chữ và số, viết hoa"""
    return re.sub(r'[^A-Za-z0-9]', '', text).upper()

def is_valid_plate(text):
    """
    Kiểm tra xem text có đúng định dạng biển số VN không.
    Quy tắc:
    1. Độ dài: Phải từ 7 ký tự trở lên (VD: 29A1234 là 7, 30A12345 là 8).
       -> Cái '123 08' của bạn sau khi clean còn '12308' (5 ký tự) -> Sẽ bị loại ngay.
    2. Cấu trúc: 2 ký tự đầu phải là số (Mã tỉnh), ký tự thứ 3 thường là Chữ.
    """
    if len(text) < 7 or len(text) > 9:
        return False

    # Kiểm tra Regex:
    # ^[0-9]{2} : Bắt đầu bằng 2 số
    # [A-Z]     : Tiếp theo là 1 chữ cái
    # [0-9]+    : Sau đó là dãy số
    # $         : Kết thúc chuỗi
    # Ví dụ khớp: 30A12345, 90B56789, 29C1234
    pattern = r"^[0-9]{2}[A-Z][0-9]+$"

    # Nếu khớp pattern thì True, không thì False
    if re.match(pattern, text):
        return True

    # Mở rộng cho biển quân đội/xanh (AA, 80B...) nếu cần thì sửa regex sau.
    # Hiện tại cứ check 2 ký tự đầu là số là an toàn nhất cho xe dân sự.
    if text[0].isdigit() and text[1].isdigit():
         return True

    return False

def format_plate(text):
    """Định dạng thêm dấu chấm/gạch ngang"""
    if len(text) == 8: return f"{text[:3]}-{text[3:6]}.{text[6:]}"
    elif len(text) == 7: return f"{text[:3]}-{text[3:]}"
    elif len(text) == 9: return f"{text[:3]}-{text[3:6]}.{text[7:]}" # Fix lỗi thừa 1 số
    return text

def process_ocr_result(ocr_result):
    if ocr_result is None: return None

    # Sắp xếp text từ trên xuống
    sorted_res = sorted(ocr_result, key=lambda x: x[0][0][1])
    raw_text = ""
    for line in sorted_res:
        text, conf = line[1]
        if conf > 0.5: raw_text += text

    cleaned = clean_text(raw_text)

    # --- ÁP DỤNG BỘ LỌC TẠI ĐÂY ---
    if is_valid_plate(cleaned):
        return format_plate(cleaned)
    else:
        return None # Trả về None nếu là rác (VD: 12308)

# ==========================================
# 2. LOAD MODELS
# ==========================================
@st.cache_resource
def load_models():
    try:
        detect_model = YOLO(MODEL_PATH)
    except:
        return None, None
    ocr_model = PaddleOCR(use_angle_cls=False, lang='en', show_log=False, use_gpu=False)
    return detect_model, ocr_model

# ==========================================
# 3. HÀM VẼ (CẬP NHẬT)
# ==========================================
def draw_results(frame, results, ocr_model):
    detected_plates = []
    h, w, _ = frame.shape

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        if float(box.conf[0]) < 0.45: continue # Tăng ngưỡng tin cậy lên chút

        crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
        if crop.shape[0] < 15 or crop.shape[1] < 15: continue # Bỏ ảnh quá bé

        try:
            res = ocr_model.ocr(crop, cls=False)
            if res and res[0]:
                # Gọi hàm xử lý (đã có bộ lọc bên trong)
                final_text = process_ocr_result(res[0])

                # Chỉ vẽ và lưu nếu final_text KHÔNG PHẢI LÀ NONE
                if final_text:
                    detected_plates.append(final_text)

                    # Vẽ màu xanh lá cây
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    (t_w, t_h), _ = cv2.getTextSize(final_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
                    cv2.rectangle(frame, (x1, y1-30), (x1+t_w, y1), (0, 255, 0), -1)
                    cv2.putText(frame, final_text, (x1, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)
        except: pass

    return frame, list(set(detected_plates))

# ==========================================
# 4. GIAO DIỆN CHÍNH
# ==========================================
st.title("📹 Hệ thống Giám sát Bãi xe (Đã lọc nhiễu)")
detect_model, ocr_model = load_models()

if detect_model is None:
    st.error(f"❌ Không tìm thấy model tại {MODEL_PATH}")
else:
    mode = st.sidebar.radio("Chọn chế độ:", ["🖼️ Hình ảnh", "🎥 Video Clip"])
    col1, col2 = st.columns([1.5, 1])

    with col1:
        st.subheader("📺 Màn hình Giám sát")

        # --- ẢNH ---
        if mode == "🖼️ Hình ảnh":
            uploaded_file = st.file_uploader("Upload ảnh:", type=['jpg', 'png'])
            if uploaded_file:
                image = Image.open(uploaded_file).convert('RGB')
                img_np = np.array(image)
                if st.button("🔍 Quét biển số", type="primary"):
                    with st.spinner("Đang phân tích..."):
                        results = detect_model(img_np, verbose=False)
                        final_img, plates = draw_results(img_np.copy(), results, ocr_model)
                        st.image(final_img, caption="Kết quả", use_container_width=True)

                        if plates:
                            p = plates[0]
                            st.success(f"🚘 Biển số hợp lệ: **{p}**")
                            now = datetime.now().strftime("%H:%M %d/%m")
                            df = pd.DataFrame([[now, p, "Ảnh", "---"]], columns=['Giờ', 'Biển số', 'Nguồn', 'Ghi chú'])
                            df.to_csv(HISTORY_FILE, mode='a', header=not os.path.exists(HISTORY_FILE), index=False)
                        else:
                            st.warning("Không tìm thấy biển số hợp lệ (Đã lọc rác).")

        # --- VIDEO ---
        elif mode == "🎥 Video Clip":
            uploaded_video = st.file_uploader("Upload video:", type=['mp4', 'mov', 'avi'])
            if uploaded_video:
                tfile = tempfile.NamedTemporaryFile(delete=False)
                tfile.write(uploaded_video.read())

                if st.button("▶️ Bắt đầu Xử lý", type="primary"):
                    cap = cv2.VideoCapture(tfile.name)
                    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    st_frame = st.empty()
                    progress_bar = st.progress(0)
                    status_text = st.empty()

                    output_path = "output_processed.mp4"
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    fps = cap.get(cv2.CAP_PROP_FPS)
                    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

                    seen_plates_in_video = set()
                    frame_count = 0

                    while cap.isOpened():
                        ret, frame = cap.read()
                        if not ret: break
                        frame_count += 1

                        results = detect_model(frame, verbose=False)
                        # Hàm draw_results giờ đây chỉ trả về biển số ĐÚNG CHUẨN
                        processed_frame, plates = draw_results(frame, results, ocr_model)
                        out.write(processed_frame)

                        st_frame.image(cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB), use_container_width=True)
                        progress_bar.progress(frame_count / total_frames)

                        if plates:
                            p = plates[0]
                            if p not in seen_plates_in_video:
                                seen_plates_in_video.add(p)
                                now = datetime.now().strftime("%H:%M %d/%m")
                                df = pd.DataFrame([[now, p, "Video", "Phát hiện mới"]], columns=['Giờ', 'Biển số', 'Nguồn', 'Ghi chú'])
                                df.to_csv(HISTORY_FILE, mode='a', header=not os.path.exists(HISTORY_FILE), index=False)
                                st.toast(f"✅ Đã ghi nhận: {p}", icon="💾")
                            status_text.success(f"⚡ Đang theo dõi: {p}")

                    cap.release()
                    out.release()
                    status_text.success("✅ Đã xử lý xong!")
                    with open(output_path, 'rb') as f:
                        st.download_button('⬇️ Tải Video', f, file_name='video_result.mp4')

    # --- LỊCH SỬ ---
    with col2:
        st.subheader("📊 Lịch sử (Đã lọc rác)")
        c_refresh, c_del = st.columns(2)
        if c_refresh.button("🔄 Làm mới"): st.rerun()
        if c_del.button("🗑️ Xóa hết"):
            if os.path.exists(HISTORY_FILE): os.remove(HISTORY_FILE)
            st.rerun()

        if os.path.exists(HISTORY_FILE):
            st.dataframe(pd.read_csv(HISTORY_FILE).iloc[::-1], use_container_width=True, hide_index=True)

Writing app.py


In [4]:
import subprocess
import time
import re
import os

# --- 1. DỌN DẸP TIẾN TRÌNH CŨ (QUAN TRỌNG) ---
# Đoạn này giúp bạn chạy lại cell này thoải mái mà không bị lỗi "Port already in use"
print("🧹 Đang tắt server cũ để khởi động lại...")
os.system("pkill -9 streamlit")
os.system("pkill -9 cloudflared")
time.sleep(2) # Đợi 2 giây cho nó tắt hẳn

# --- 2. TẢI CÔNG CỤ (Chỉ tải nếu chưa có) ---
if not os.path.exists("cloudflared-linux-amd64"):
    print("⬇️ Đang tải Cloudflare Tunnel...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared-linux-amd64

# --- 3. CHẠY STREAMLIT NGẦM ---
print("🚀 Đang bật Streamlit App...")
# Chạy streamlit dưới nền
os.system("nohup streamlit run app.py > /dev/null 2>&1 &")
time.sleep(3) # Đợi Streamlit khởi động xong

# --- 4. MỞ CỔNG VÀ LẤY LINK ---
print("🔗 Đang tạo đường hầm kết nối...")

# Xóa log cũ để đảm bảo lấy đúng link mới
if os.path.exists('cf_log.txt'):
    os.remove('cf_log.txt')

# Chạy Cloudflare Tunnel
with open('cf_log.txt', 'w') as log_file:
    process = subprocess.Popen(['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:8501'], stdout=log_file, stderr=log_file)

# Quét log để tìm link
cf_url = ""
start_time = time.time()
while True:
    # Nếu đợi quá 20s mà không thấy link thì báo lỗi
    if time.time() - start_time > 20:
        print("\n⚠️ Có vẻ Streamlit khởi động chậm hoặc bị lỗi code. Hãy kiểm tra lại file app.py")
        break

    if os.path.exists('cf_log.txt'):
        with open('cf_log.txt', 'r') as f:
            content = f.read()
            match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
            if match:
                cf_url = match.group(0)
                break
    time.sleep(1)
    print(".", end="")

if cf_url:
    print("\n\n" + "="*40)
    print("✅ RESET THÀNH CÔNG!")
    print("👉 Link truy cập mới của bạn đây:")
    print(f"   {cf_url}")
    print("="*40)

🧹 Đang tắt server cũ để khởi động lại...
⬇️ Đang tải Cloudflare Tunnel...
🚀 Đang bật Streamlit App...
🔗 Đang tạo đường hầm kết nối...
.....

✅ RESET THÀNH CÔNG!
👉 Link truy cập mới của bạn đây:
   https://web-dreams-felt-mia.trycloudflare.com
